In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('/kaggle/input/should-this-loan-be-approved-or-denied/SBAnational.csv')
df.head()

/tmp/ipykernel_17/3663163007.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/kaggle/input/should-this-loan-be-approved-or-denied/SBAnational.csv')


,LoanNr_ChkDgt,Name,City,State,Zip,Bank,BankState,NAICS,ApprovalDate,ApprovalFY,...,RevLineCr,LowDoc,ChgOffDate,DisbursementDate,DisbursementGross,BalanceGross,MIS_Status,ChgOffPrinGr,GrAppv,SBA_Appv
0,1000014003,ABC HOBBYCRAFT,EVANSVILLE,IN,47711,FIFTH THIRD BANK,OH,451120,28-Feb-97,1997,...,N,Y,NaN,28-Feb-99,"$60,000.00",$0.00,P I F,$0.00,"$60,000.00","$48,000.00"
1,1000024006,LANDMARK BAR & GRILLE (THE),NEW PARIS,IN,46526,1ST SOURCE BANK,IN,722410,28-Feb-97,1997,...,N,Y,NaN,31-May-97,"$40,000.00",$0.00,P I F,$0.00,"$40,000.00","$32,000.00"
2,1000034009,"WHITLOCK DDS, TODD M.",BLOOMINGTON,IN,47401,GRANT COUNTY STATE BANK,IN,621210,28-Feb-97,1997,...,N,N,NaN,31-Dec-97,"$287,000.00",$0.00,P I F,$0.00,"$287,000.00","$215,250.00"
3,1000044001,"BIG BUCKS PAWN & JEWELRY, LLC",BROKEN ARROW,OK,74012,1ST NATL BK & TR CO OF BROKEN,OK,0,28-Feb-97,1997,...,N,Y,NaN,30-Jun-97,"$35,000.00",$0.00,P I F,$0.00,"$35,000.00","$28,000.00"
4,1000054004,"ANASTASIA CONFECTIONS, INC.",ORLANDO,FL,32801,FLORIDA BUS. DEVEL CORP,FL,0,28-Feb-97,1997,...,N,N,NaN,14-May-97,"$229,000.00",$0.00,P I F,$0.00,"$229,000.00","$229,000.00"


In [3]:
# create dummy variable RealEstate - calculated and deemed important based on the findings of the research paper attached
# checked dtype, min and max of Term beforehand
df['RealEstate'] = df['Term']>=240

In [4]:
# create dummy variable Recession - calculated and deemed important based on the findings of the research paper attached
# checked dtype, min and max of DisbursementDate beforehand
df['DisbursementDate'] = pd.to_datetime(df['DisbursementDate'], format='%d-%b-%y')
# edit future dates to be in the 90s
df['DisbursementDate'] = df['DisbursementDate'].apply(lambda x: x if x.year < 2024 else x.replace(year=x.year-100))
df['Recession'] = (df['DisbursementDate'] >= pd.to_datetime('2007-12-01')) & (df['DisbursementDate'] < pd.to_datetime('2009-07-01'))

In [5]:
# drop most recent loans according to research paper claims of incomplete data
df = df[df['DisbursementDate'] <= '2010-12-31']

In [6]:
# create dummy variable Default
# print(df['MIS_Status'].value_counts()) # only 2 types of values, good to go
df['Default'] = df['MIS_Status']=='CHGOFF' #TODO not good

In [7]:
#drop irrelevant columns, based on personal notes and study
columns_to_drop = ['LowDoc', 'CreateJob', 'RetainedJob', 'FranchiseCode', 'DisbursementDate', 'LoanNr_ChkDgt', 'Name', 'Zip', 'ApprovalDate', 'ApprovalFY', 'NewExist', 'ChgOffDate', 'DisbursementGross', 'BalanceGross', 'ChgOffPrinGr', 'SBA_Appv', 'MIS_Status']
df.drop(columns=columns_to_drop, inplace=True)

In [8]:
#clean data and format remaining columns
df = df[df['NAICS'] != 0]
df['NAICS'] = df['NAICS'].astype(str).str[:2].astype(int)
df = df[df['RevLineCr'].isin(['N', 'Y'])]

In [9]:
df.dropna(inplace=True)
df.head()

,City,State,Bank,BankState,NAICS,Term,NoEmp,UrbanRural,RevLineCr,GrAppv,RealEstate,Recession,Default
0,EVANSVILLE,IN,FIFTH THIRD BANK,OH,45,84,4,0,N,"$60,000.00",False,False,False
1,NEW PARIS,IN,1ST SOURCE BANK,IN,72,60,2,0,N,"$40,000.00",False,False,False
2,BLOOMINGTON,IN,GRANT COUNTY STATE BANK,IN,62,180,7,0,N,"$287,000.00",False,False,False
5,PLAINVILLE,CT,"TD BANK, NATIONAL ASSOCIATION",DE,33,120,19,0,N,"$517,000.00",False,False,False
7,SUMMERFIELD,FL,REGIONS BANK,AL,81,84,1,0,N,"$45,000.00",False,False,False


In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

X = df.drop(columns=['Default'])
y = df['Default']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

numeric_features = X_train.select_dtypes(include=['int64', 'float64']).columns
numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])

categorical_features = X_train.select_dtypes(include=['object']).columns
categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', LogisticRegression(max_iter=1000))])

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))
print("\nAccuracy Score:", accuracy_score(y_test, y_pred))

Confusion Matrix:
[[65784  3358]
 [ 7403 12345]]

Classification Report:
              precision    recall  f1-score   support

       False       0.90      0.95      0.92     69142
        True       0.79      0.63      0.70     19748

    accuracy                           0.88     88890
   macro avg       0.84      0.79      0.81     88890
weighted avg       0.87      0.88      0.87     88890


Accuracy Score: 0.8789402632467094
